In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [2]:
# Some constants for the model

# R: hospitalization rates.
# 2x3 (age group x dose group)
# first row for ppl below 65 and second row for ppl above 65
# first to third column: 1, 2, and 0 doses of vaccine
R = Matrix([0.003 0.002 0.03; 0.015 0.01 0.15]) 
gamma = 0.1 # State can only vaccinate 5% of population per week

0.1

In [3]:
data_folder = "real_world_data"
# Load csv files
case_file_g1 = data_folder * "/" * "case_data_g1.csv"
case_file_g2 = data_folder * "/" * "case_data_g2.csv"
population_file = data_folder * "/" * "population_data_age.csv"
vaccine_file = data_folder * "/" * "vaccine_data.csv"

case_data_g1 = CSV.read(case_file_g1, DataFrame)
case_data_g2 = CSV.read(case_file_g2, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)

P = Matrix(population_data)
CG1 = Matrix(case_data_g1)
CG2 = Matrix(case_data_g2)
V = Matrix(vaccine_data)
# convert population data to matrix

CG1 = CG1'
CG2 = CG2'
C = cat(CG1, CG2, dims=3)
P = P[:,2:3]
V = V[:,1]

println("C: ", size(C))
println("P: ", size(P))
println("V: ", size(V))

n_states = 1:size(case_data_g1, 2)
n_weeks = 1:size(case_data_g1, 1) 
n_weeks_2 = 1:size(case_data_g1, 1)-1 

n_groups = 1:size(P, 2)
n_doses = 1:3

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)
println("Number of groups: ", n_groups)
println("Number of dose groups: ", n_doses)

C: (50, 52, 2)
P: (50, 2)
V: (52,)
Number of states: 1:50
Number of weeks: 1:52
Number of groups: 1:2
Number of dose groups: 1:3


In [4]:
model = Model(Gurobi.Optimizer)

@variable(model, X[n_states, n_weeks, n_groups, n_doses] >= 0);
@variable(model, W[n_states, n_weeks, n_groups, n_doses] >= 0);
@variable(model, Y[n_states, n_weeks] >= 0);

@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks));

@constraint(model, hospitalizations[i in n_states, j in n_weeks],
       Y[i,j] == sum(sum(C[i,j,g] * W[i,j,g,d] * R[g,d] for g in n_groups) for d in n_doses))

@constraint(model, initial_vax_frac[i in n_states, g in n_groups, d in 1:2],
       W[i,1,g,d] == 0) 

@constraint(model, frac_bounds[i in n_states, j in n_weeks, g in n_groups, d in n_doses],
       W[i,j,g,d] <= 1)

@constraint(model, vax1_frac[i in n_states, j in n_weeks_2, g in n_groups],
       W[i,j+1,g,1] == W[i,j,g,1] + (X[i,j+1,g,1] - X[i,j+1,g,2]) / P[i,g]) 

@constraint(model, vax2_frac[i in n_states, j in n_weeks_2, g in n_groups],
       W[i,j+1,g,2] == W[i,j,g,2] + X[i,j+1,g,2] / P[i,g]) 

@constraint(model, novax_frac[i in n_states, j in n_weeks, g in n_groups],
       W[i,j,g,3] == 1 - W[i,j,g,1] - W[i,j,g,2])

@constraint(model, num_vax2_c1[i in n_states, j in 1:5, g in n_groups],
       X[i,j,g,2] == 0) 

@constraint(model, num_vax2_c2[i in n_states, j in 6:52, g in n_groups],
       X[i,j,g,2] <= sum(X[i,k,g,1] for k in 1:j-5) - sum(X[i,k,g,2] for k in 1:j-1)) 

@constraint(model, vax_supply[j in n_weeks],
       sum(sum(sum(X[i,j,g,d] for i in n_states) for g in n_groups) for d in n_doses) <= V[j]) 

@constraint(model, state_vax_capacity[i in n_states, j in n_weeks],
       sum(sum(X[i,j,g,d] for g in n_groups) for d in n_doses) <= gamma * (P[i,1] + P[i,2]));

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-18


In [5]:
optimize!(model)
objective_value(model)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 41652 rows, 33800 columns and 366100 nonzeros
Model fingerprint: 0x2152dc46
Coefficient statistics:
  Matrix range     [3e-08, 8e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+07]
Presolve removed 23800 rows and 13400 columns
Presolve time: 0.05s
Presolved: 17852 rows, 20400 columns, 290100 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 6.692e+05
 Factor NZ  : 1.048e+06 (roughly 24 MB of memory)
 Factor Ops : 7.472e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.78739075e+07 -5.12206514e+07  1.4

530267.8351419154

In [6]:
W = value.(W)
X = value.(X)
Y = value.(Y);

X_soln = zeros(total_states, total_weeks,2,2)
Y_soln = zeros(total_states, total_weeks)
W_soln = zeros(total_states, total_weeks,2,2)
# Compute distances between all pairs of points
for i in n_states
    for j in n_weeks
        Y_soln[i,j] = Y[i,j]
        for g in 1:2
            for d in 1:2
                X_soln[i,j,g,d] = X[i,j,g,d]
                W_soln[i,j,g,d] = W[i,j,g,d]
            end
        end
    end
end

# save the solutions as a jld file
save("real_soln_DD_AG.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)